In [1]:
from neomaril_codex.training import NeomarilTrainingClient

In [2]:
client = NeomarilTrainingClient()
client

2023-11-09 11:52:03.504 | INFO     | neomaril_codex.training:__init__:1063 - Loading .env
2023-11-09 11:52:03.505 | INFO     | neomaril_codex.base:__init__:90 - Loading .env
2023-11-09 11:52:05.003 | INFO     | neomaril_codex.base:__init__:102 - Successfully connected to Neomaril


NeomarilTrainingClient(url="http://localhost:7070/api", version="eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlFnc0JWQ0I5WFc0V1YtSkVCVkJiZyJ9.eyJodHRwczovL25lb21hcmlsLmRhdGFyaXNrLm5ldC9uZW9tYXJpbC1ncm91cCI6ImRhdGFyaXNrIiwiaXNzIjoiaHR0cHM6Ly9kZXYtbWszbzdsYXp4bGUzMGh3cS51cy5hdXRoMC5jb20vIiwic3ViIjoiYXV0aDB8NjU0OTRlMWFkOTUzN2FlMGFhZDZjNGE5IiwiYXVkIjpbImh0dHBzOi8vZGV2LW1rM283bGF6eGxlMzBod3EudXMuYXV0aDAuY29tL2FwaS92Mi8iLCJodHRwczovL2Rldi1tazNvN2xhenhsZTMwaHdxLnVzLmF1dGgwLmNvbS91c2VyaW5mbyJdLCJpYXQiOjE2OTk1NDE1MjQsImV4cCI6MTY5OTU1MjMyNCwiYXpwIjoia3JCNk1sR3ZkOEdBSUNZd1hPd0labU5DMFk5VGFQQTciLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGFkZHJlc3MgcGhvbmUgcmVhZDpjdXJyZW50X3VzZXIgdXBkYXRlOmN1cnJlbnRfdXNlcl9tZXRhZGF0YSBkZWxldGU6Y3VycmVudF91c2VyX21ldGFkYXRhIGNyZWF0ZTpjdXJyZW50X3VzZXJfbWV0YWRhdGEgY3JlYXRlOmN1cnJlbnRfdXNlcl9kZXZpY2VfY3JlZGVudGlhbHMgZGVsZXRlOmN1cnJlbnRfdXNlcl9kZXZpY2VfY3JlZGVudGlhbHMgdXBkYXRlOmN1cnJlbnRfdXNlcl9pZGVudGl0aWVzIG9mZmxpbmVfYWNjZXNzIiwiZ3R5IjoicGFzc3dvcmQifQ.SY_-1w7iUrOny_dnMzV

In [3]:
training = client.create_training_experiment('Teste', 'Classification', group='datarisk')

2023-11-09 11:52:05.036 | INFO     | neomaril_codex.training:create_training_experiment:1164 - New Training 'Teste' inserted.


2023-11-09 11:52:05.037 | INFO     | neomaril_codex.training:__init__:611 - Loading .env


In [4]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [5]:
base_path = './samples/train/'

In [6]:
df = pd.read_csv(base_path+"/dados.csv")

In [7]:
X = df.drop(columns=['target'])
y = df[["target"]]

In [8]:
pipe = make_pipeline(SimpleImputer(), LGBMClassifier(force_col_wise=True))

In [9]:
with training.log_train('Teste 1', X, y) as logger:
    pipe.fit(X, y)
    logger.save_model(pipe)

    logger.add_extra('./extra.txt')
    
    params = pipe.get_params()

    model_output = pd.DataFrame({"pred": pipe.predict(X), "proba": pipe.predict_proba(X)[:,1]})
    logger.save_model_output(model_output)

    auc = cross_val_score(pipe, X, y, cv=5, scoring="roc_auc")
    f_score = cross_val_score(pipe, X, y, cv=5, scoring="f1")
    logger.save_metric(name='auc', value=auc.mean())
    logger.save_metric(name='f1_score', value=f_score.mean())

    logger.set_python_version('3.10')

/home/jonathan-carvalho/.local/share/virtualenvs/mlops-neomaril-codex-c4z0dHNl/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/jonathan-carvalho/.local/share/virtualenvs/mlops-neomaril-codex-c4z0dHNl/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/jonathan-carvalho/.local/share/virtualenvs/mlops-neomaril-codex-c4z0dHNl/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = 

X_train:./Teste_1/features.parquet
y_train:./Teste_1/target.parquet
model_outputs:./Teste_1/predictions.parquet
model_file:./Teste_1/model.pkl
model_metrics:./Teste_1/metrics.json
logger.info({"ExecutionId":2,"Message":"External training files have been partially uploaded, in order to use all features, please upload all training files! Use the id \u00272\u0027 to execute the train experiment."}


/home/jonathan-carvalho/.local/share/virtualenvs/mlops-neomaril-codex-c4z0dHNl/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/jonathan-carvalho/.local/share/virtualenvs/mlops-neomaril-codex-c4z0dHNl/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/jonathan-carvalho/.local/share/virtualenvs/mlops-neomaril-codex-c4z0dHNl/lib/python3.10/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") 